In [1]:
!nvidia-smi

Mon Oct 13 11:51:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.05              Driver Version: 560.35.05      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        On  |   00000000:81:00.0 Off |                  N/A |
|  0%   39C    P8             31W /  370W |       2MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!kill 939297

/bin/bash: line 1: kill: (939297) - No such process


In [2]:
import sys, torch
print("Python:", sys.version)
print("Executable:", sys.executable)
print("CUDA available:", torch.cuda.is_available())


Python: 3.10.18 | packaged by conda-forge | (main, Jun  4 2025, 14:45:41) [GCC 13.3.0]
Executable: /upb/users/b/balram/profiles/unix/cs/.conda/envs/kg_pipeline/bin/python3
CUDA available: True


In [3]:
# Block 0 — Imports & Config

import os, json, re, time
import torch
from textwrap import dedent
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [4]:
def setup_model(model_id="mistralai/Mistral-7B-Instruct-v0.3"):
    print("⏳ Loading model:", model_id)
    torch.backends.cudnn.benchmark = True

    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype=torch.float16,
    )
    model.config.use_cache = True  # KV caching is fine; doesn't cause "same output"

    generator = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        device_map="auto"
    )
    #print("✅ Model loaded successfully.")
    return generator, tokenizer


In [5]:
# ---------- Utilities ----------

def _quote(v: object) -> str:
    """
    Convert any value to a safe, double-quoted string for the triple format.
    Escapes backslashes and double quotes.
    """
    s = str(v)
    s = s.replace("\\", "\\\\").replace('"', '\\"')
    return f'"{s}"'


def _coerce_triples_to_str(triples_val):
    """
    Normalize triples into newline-separated lines: rel("Subject","Object")

    Accepts:
      - str: a preformatted block
      - list[str]: each a preformatted line
      - list[dict]: each with keys {sub, rel, obj}
    """
    if triples_val is None:
        return ""

    # Already a full block string
    if isinstance(triples_val, str):
        return triples_val.strip()

    # List of preformatted strings
    if isinstance(triples_val, list) and all(isinstance(x, str) for x in triples_val):
        return "\n".join(x.strip() for x in triples_val if x and x.strip())

    # List of dicts -> format to rel("sub","obj")
    if isinstance(triples_val, list) and all(isinstance(x, dict) for x in triples_val):
        lines = []
        for t in triples_val:
            rel = (t.get("rel") or "").strip()
            sub = t.get("sub")
            obj = t.get("obj")
            if rel and (sub is not None) and (obj is not None):
                lines.append(f'{rel}({_quote(sub)},{_quote(obj)})')
        return "\n".join(lines)

    # Fallback
    return str(triples_val).strip()


# ---------- Few-shot loader for FIXED schema ----------

def load_one_shot_examples(few_shot_jsonl_path: str) -> dict:
    """
    Loads {id: {"example_sentence": <str>, "example_triples_output": <any>}}.
    - Reads fields exactly as written in your file:
        "Example sentence"
        "Example triples output"   (and falls back to "Example triples" if the former is absent)
    - No validation / coercion.
    """
    out = {}
    for rec in read_jsonl(few_shot_jsonl_path, max_items=None):
        rid = rec.get("id")
        if rid is None:
            continue  # keep this tiny guard so the dict key isn't None
        out[str(rid).strip()] = {
            "example_sentence": rec.get("Example sentence"),
            # prefer "Example triples output", else fallback to "Example triples"
            "example_triples_output": rec.get("Example triples output", rec.get("Example triples")),
        }
    print(f"[FEW-SHOT] Loaded {len(out)} examples from: {few_shot_jsonl_path}")
    return out



In [16]:
# Block 2 — Prompt Builder (Reason → Verify → Final Triples)

from textwrap import dedent
import json

# ---------- Block 2 — Prompt Builder (Reason → Verify → Final Triples) ----------

# Helpers (strict access; fail fast if ontology keys are missing)
def _concept_label(ontology_json, qid):
    return next((c["label"] for c in ontology_json["concepts"] if c["qid"] == qid), "")

def format_ontology_concepts(ontology_json):
    return ", ".join(c["label"] for c in ontology_json["concepts"])

def format_ontology_relations(ontology_json):
    lines = []
    for r in ontology_json["relations"]:
        dom = _concept_label(ontology_json, r["domain"])
        rng = _concept_label(ontology_json, r["range"])
        lines.append(f'- {r["label"]}({dom},{rng})')
    return "\n".join(lines)


def build_reason_then_extract_prompt(
    ontology_json,
    test_sentence,
    worked_example=None,
    allow_light_norm=True,
    relation_cues_text=None,   # optional Step 0 paraphrase cues
    strict_format=True,        # enforce exact output shape
):
    """
    Build a prompt that ends with the header `### FINAL TRIPLES` and *no trailing guidance*,
    so generations appear directly under the header in the required line format.

    This version matches the downstream extractor which searches for `##/### FINAL TRIPLES`.
    """

    concepts_line   = format_ontology_concepts(ontology_json)
    relations_block = format_ontology_relations(ontology_json)
    norm_note = (
        "normalize only trivial cases (e.g., Japanese→Japan)"
        if allow_light_norm else
        "copy spans verbatim; no normalization"
    )

    # Optional Step 0 cues
    step0_block = ""
    if relation_cues_text:
        step0_block = dedent(f"""
        Step 0 (Paraphrase expansion for this sentence):
        {relation_cues_text.strip()}
        """)

    # Optional worked example — supports either the new shape or the older example fields
    example_block = ""
    if worked_example:
        ex_sent = worked_example.get("example_sentence")
        ex_out  = worked_example.get("example_triples_output")
        #         # Use JSON dumps to preserve quotes/escapes for the sentence,
        # and to serialize lists/dicts as-is for the output block.
        ex_sent_json = json.dumps(ex_sent, ensure_ascii=False)  # -> "The number of staff..."
        ex_out_json  = json.dumps(ex_out, ensure_ascii=False)   # -> [{"sub":...,"rel":...,"obj":...}, ...]
        example_block = dedent(f'''
        "Example sentence": {ex_sent_json}

        "Example FINAL TRIPLES": {ex_out_json}
        ''').strip()

    # Strict output notes merged into RULES (no trailing text after the header in the final prompt)
    strict_notes = ""
    if strict_format:
        strict_notes = dedent("""
        OUTPUT SHAPE (strict):
        - After Step 2, output the header exactly:
        ### FINAL TRIPLES
        - Then output one triple per line format: predicate("Subject","Object")
        - No extra commentary after the triples. If no triples, just output the header.
        - Do NOT output JSON, code fences, or any other sections.
        """).strip()

    rules = dedent(f"""
    RULES:
    - Prefer relation labels from ONTOLOGY RELATIONS when they match the evidence.
    - If the sentence clearly states another SPO relation that is NOT in the ontology, still include it using a concise predicate phrase derived from the trigger words in the sentence (do NOT invent facts).
    - Resolve simple coreference (e.g., "the film", "this movie", "it" → the film title in this sentence).
    - Quote/mention evidence in Step 2 for each emitted triple.
    - Base decisions ONLY on the TEST SENTENCE.
    - Avoid duplicates; apply light normalization only (e.g., Japanese→Japan).
    {strict_notes}
    """).strip()

    # IMPORTANT: end the prompt with the header and NOTHING after it.
    prompt = dedent(f"""
    TASK: Extract all SPO triples that the TEST SENTENCE clearly supports.
    - Use the ontology to guide relation labeling when possible.
    - {norm_note}.
    - If a valid SPO fact has no matching ontology label, still output it (single FINAL TRIPLES list).

    ONTOLOGY CONCEPTS:
    {concepts_line}

    ONTOLOGY RELATIONS (argument types):
    {relations_block}

    {rules}

    {step0_block}

    {example_block if example_block else ""}

    ### TEST SENTENCE
    "{test_sentence}"

    Step 1 (Entities & types):
    Step 2 (Verify relations): For each candidate triple, quote or mention the trigger phrase and spans.

    ### FINAL TRIPLES
    """).strip()

    return prompt


In [17]:
# Block 3 — Single Inference (chat template; continuation-only)

def generate_triples_text(generator, tokenizer, prompt_text: str,
                          max_new_tokens: int = 768, temperature: float = 0.3) -> str:
    """
    Calls the model once. Returns the full generated continuation (reasoning + FINAL TRIPLES).
    """
    chat = [
        {"role": "system", "content": "You are a precise information-extraction model. Follow instructions carefully."},
        {"role": "user", "content": prompt_text}
    ]
    formatted = tokenizer.apply_chat_template(chat, tokenize=False, add_generation_prompt=True)

    out = generator(
        formatted,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_p=0.9,
        do_sample=True,
        return_full_text=False,
        truncation=False,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
    )
    return out[0]["generated_text"] if isinstance(out[0], dict) else out[0]


In [18]:
# Block 4 — Extract & Parse `FINAL TRIPLES`

FINAL_HEADER_REGEX = r'^\s*#{2,}\s*FINAL\s+TRIPLES\s*$'  # matches ## or ### FINAL TRIPLES

def extract_final_triples_block(model_output_text: str) -> str:
    # try normal header first
    #print("model_output_text: ",model_output_text)
    
    header = re.search(FINAL_HEADER_REGEX, model_output_text, re.IGNORECASE | re.MULTILINE)
    if header:
        start = header.end()
        tail = model_output_text[start:].strip()
        nxt = re.search(r'^\s*#{2,}\s+[A-Z].*$', tail, re.MULTILINE)
        return tail[:nxt.start()].strip() if nxt else tail

    # fallback: take everything and let the parser fish valid lines out
    return model_output_text

def parse_triples_block(block_text: str):
    """
    Parse lines of the form:
        predicate("Subject","Object")
        predicate('Subject','Object')
        predicate(Subject,Object)
        predicate(Subject, 250)
    Keeps both ontology and non-ontology predicates (open-set allowed).
    Returns a list of dicts: {"sub": ..., "rel": ..., "obj": ...}
    """
    triples = []

    line_re = re.compile(r"""
    ^\s*(?:[-*•]\s*)?                 # optional markdown bullet
    (?P<rel>[A-Za-z][A-Za-z0-9_ ]*?)  # predicate name
    \s*\(\s*
    (?:                               # SUBJECT (quoted or unquoted)
        ["'](?P<sub_q>[^"']+?)["']    # "Subject" or 'Subject'
        | (?P<sub_u>[^,)\n]+?)        # Subject (unquoted)
    )
    \s*,\s*
    (?:                               # OBJECT (quoted or unquoted)
        ["'](?P<obj_q>[^"']+?)["']    # "Object" or 'Object'
        | (?P<obj_u>[^)\n]+?)         # Object (unquoted)
    )
    \s*\)\s*\.?\s*$                   # optional trailing period
""", re.VERBOSE)


    for raw in block_text.splitlines():
        line = raw.strip()
        if not line or line.startswith("#"):
            continue

        m = line_re.match(line)
        if not m:
            # skip non-matching lines quietly
            continue

        rel = m.group("rel").strip()
        sub = (m.group("sub_q") or m.group("sub_u") or "").strip()
        obj = (m.group("obj_q") or m.group("obj_u") or "").strip()

        # Optional: guard against empties
        if not rel or not sub or not obj:
            continue

        triples.append({"sub": sub, "rel": rel, "obj": obj})

    return triples


In [19]:
# Block 5 — JSONL I/O Helpers

def read_jsonl(path, max_items: int | None = None):
    """
    Yields JSON objects from a .jsonl file.
    If max_items is set, stops after that many records (for debugging).
    """
    count = 0
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            yield json.loads(line)
            count += 1
            if max_items is not None and count >= max_items:
                break

def write_jsonl(path, records):
    with open(path, "w", encoding="utf-8") as f:
        for rec in records:
            f.write(json.dumps(rec, ensure_ascii=False))
            f.write("\n")


In [23]:
# Block 6 — Orchestrator (loop inputs → Steps 1–4 → output)

def run_pipeline(
    input_jsonl_path: str,
    ontology_json_path: str,
    output_jsonl_path: str,
    max_items: int = 4,
    max_new_tokens: int = 768,
    temperature: float = 0.25,
    verbose: bool = True,
    few_shot_jsonl_path: str | None = None,   # pass as keyword in the call below
):
    """
    Runs the extraction pipeline.
    Required: input_jsonl_path, ontology_json_path, output_jsonl_path.
    Optional: max_items, max_new_tokens, temperature, verbose, few_shot_jsonl_path.
    """
    # ✅ Load ontology once
    with open(ontology_json_path, "r", encoding="utf-8") as f:
        ontology = json.load(f)

    # ✅ Load one-shot examples (if provided)
    if few_shot_jsonl_path:
        one_shot_by_id = load_one_shot_examples(few_shot_jsonl_path)
    else:
        one_shot_by_id = {}
        print("[FEW-SHOT] No few_shot_jsonl_path provided — proceeding without examples.")

    # ✅ Setup model once
    generator, tokenizer = setup_model("mistralai/Mistral-7B-Instruct-v0.3")

    out_records = []
    t0 = time.time()

    # ✅ Iterate inputs
    for idx, item in enumerate(read_jsonl(input_jsonl_path, max_items=max_items), start=1):
        sent_id = item.get("id")
        sent    = item.get("sent", "")

        # Match example by id
        worked_example = one_shot_by_id.get(sent_id)
        # Build prompt
        prompt_text = build_reason_then_extract_prompt(
            ontology_json=ontology,
            test_sentence=sent,
            worked_example=worked_example,
            allow_light_norm=True,
            relation_cues_text=None
        )

        print("prompt_text:",prompt_text)


        # Generate
        t_gen0 = time.time()
        model_output = generate_triples_text(
            generator, tokenizer, prompt_text,
            max_new_tokens=max_new_tokens, temperature=temperature
        )
        gen_time = time.time() - t_gen0

        # Extract & Parse
        final_block = extract_final_triples_block(model_output)

        triples = parse_triples_block(final_block)

        out_records.append({"id": sent_id, "sentence": sent, "triples": triples})

    # Write output
    write_jsonl(output_jsonl_path, out_records)
    print(f"\n✅ Done. Wrote {len(out_records)} lines to: {output_jsonl_path} | Total time: {time.time()-t0:.1f}s")

    return out_records


In [26]:
# Block 7 — Example Run (edit these paths)

INPUT_JSONL   = "/upb/users/b/balram/profiles/unix/cs/promptKG/data/input/wikidata/input_text/ont_1_movie_test.jsonl"     # your 3 sample lines as JSONL
ONTOLOGY_JSON = "/upb/users/b/balram/profiles/unix/cs/promptKG/data/input/wikidata/input_ontology/1_movie_ontology.json"         # your ontology JSON
OUTPUT_JSONL  = "/upb/users/b/balram/profiles/unix/cs/promptKG/data/output/prompt1/wikidata/ont_1_movie_output_test.jsonl"             # where to write results
FEW_SHOT_JSONL = "/upb/users/b/balram/profiles/unix/cs/promptKG/data/fewshots_example/wikidata/ont_1_movie_few_shot.jsonl"
MAX_ITEMS = 1#None   # change to 20 (or None for full file) to control batches during debugging

In [27]:
out = run_pipeline(
    input_jsonl_path=INPUT_JSONL,
    ontology_json_path=ONTOLOGY_JSON,
    output_jsonl_path=OUTPUT_JSONL,
    max_items=MAX_ITEMS,
    max_new_tokens=768,
    temperature=0.25,
    verbose=True,
    few_shot_jsonl_path=FEW_SHOT_JSONL,  # <-- pass by keyword
)


[FEW-SHOT] Loaded 840 examples from: /upb/users/b/balram/profiles/unix/cs/promptKG/data/fewshots_example/wikidata/ont_1_movie_few_shot.jsonl
⏳ Loading model: mistralai/Mistral-7B-Instruct-v0.3


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0


prompt_text: TASK: Extract all SPO triples that the TEST SENTENCE clearly supports.
    - Use the ontology to guide relation labeling when possible.
    - normalize only trivial cases (e.g., Japanese→Japan).
    - If a valid SPO fact has no matching ontology label, still output it (single FINAL TRIPLES list).

    ONTOLOGY CONCEPTS:
    human, city, country, film, film genre, genre, film production company, film award, award, written work, film character, film organization

    ONTOLOGY RELATIONS (argument types):
    - director(film,human)
- screenwriter(film,human)
- genre(film,genre)
- based on(film,written work)
- cast member(film,human)
- award received(film,award)
- production company(film,film production company)
- country of origin(film,country)
- publication date(film,)
- characters(film,film character)
- narrative location(film,city)
- filming location(film,city)
- main subject(film,)
- nominated for(film,award)
- cost(film,)

    RULES:
    - Prefer relation labels from ONTO

In [13]:
# import os, re

# # ---- Fixed base paths (unchanged) ----
# BASE_INPUT = "/upb/users/b/balram/profiles/unix/cs/promptKG/data/input/dbpedia/input_text/"
# BASE_ONTO  = "/upb/users/b/balram/profiles/unix/cs/promptKG/data/input/dbpedia/input_ontology/"
# BASE_OUT   = "/upb/users/b/balram/profiles/unix/cs/promptKG/data/output/prompt1/dbpedia/"
# BASE_FEW   = "/upb/users/b/balram/profiles/unix/cs/promptKG/data/fewshots_example/dbpedia/"

# # ---- The 19 filenames from your screenshot ----
# FILENAMES = [
#     "ont_1_university_test.jsonl",
#     "ont_10_comicscharacter_test.jsonl",
#     "ont_11_meanoftransportation_test.jsonl",
#     "ont_12_monument_test.jsonl",
#     "ont_13_food_test.jsonl",
#     "ont_14_writtenwork_test.jsonl",
#     "ont_15_sportsteam_test.jsonl",
#     "ont_16_city_test.jsonl",
#     "ont_17_artist_test.jsonl",
#     "ont_18_scientist_test.jsonl",
#     "ont_19_film_test.jsonl",
#     "ont_2_musicalwork_test.jsonl",
#     "ont_3_airport_test.jsonl",
#     "ont_4_building_test.jsonl",
#     "ont_5_athlete_test.jsonl",
#     "ont_6_politician_test.jsonl",
#     "ont_7_company_test.jsonl",
#     "ont_8_celestialbody_test.jsonl",
#     "ont_9_astronaut_test.jsonl",
# ]

# # ont_{index}_{category}_test.jsonl  ->  {index}_{category}_ontology.json, ont_{index}_{category}_few_shot.jsonl, ont_{index}_{category}_output.jsonl
# PATTERN = re.compile(r"^ont_(\d+)_([a-z]+)_test\.jsonl$")

# def make_paths(filename: str):
#     m = PATTERN.match(filename)
#     if not m:
#         raise ValueError(f"Unexpected filename format: {filename}")
#     idx, cat = m.groups()

#     input_jsonl = os.path.join(BASE_INPUT, filename)
#     ontology_json = os.path.join(BASE_ONTO, f"{idx}_{cat}_ontology.json")
#     few_shot_jsonl = os.path.join(BASE_FEW, f"ont_{idx}_{cat}_few_shot.jsonl")

#     # ont_{idx}_{cat}_test.jsonl -> ont_{idx}_{cat}_output.jsonl
#     out_name = filename.replace("_test.jsonl", "_output.jsonl")
#     output_jsonl = os.path.join(BASE_OUT, out_name)

#     return input_jsonl, ontology_json, few_shot_jsonl, output_jsonl, f"ont_{idx}_{cat}"

# # ---- Run all files ----
# for fname in FILENAMES:
#     try:
#         INPUT_JSONL, ONTOLOGY_JSON, FEW_SHOT_JSONL, OUTPUT_JSONL, tag = make_paths(fname)

#         print("\n" + "="*80)
#         print(f"[RUN] {tag}")
#         print("INPUT :", INPUT_JSONL)
#         print("ONTO  :", ONTOLOGY_JSON)
#         print("FEWS  :", FEW_SHOT_JSONL)
#         print("OUTPUT:", OUTPUT_JSONL)

#         out = run_pipeline(
#             input_jsonl_path=INPUT_JSONL,
#             ontology_json_path=ONTOLOGY_JSON,
#             output_jsonl_path=OUTPUT_JSONL,
#             max_items=None,          # keep as-is; change if you want to limit
#             max_new_tokens=768,
#             temperature=0.25,
#             verbose=True,
#             few_shot_jsonl_path=FEW_SHOT_JSONL,
#         )

#         print(f"[DONE] {tag}")
#     except Exception as e:
#         print(f"[ERROR] {fname}: {e}")


In [11]:
import os, re

# ---- Fixed base paths (unchanged) ----
BASE_INPUT = "/upb/users/b/balram/profiles/unix/cs/promptKG/data/input/wikidata/input_text/"
BASE_ONTO  = "/upb/users/b/balram/profiles/unix/cs/promptKG/data/input/wikidata/input_ontology/"
BASE_OUT   = "/upb/users/b/balram/profiles/unix/cs/promptKG/data/output/prompt1/wikidata/"
BASE_FEW   = "/upb/users/b/balram/profiles/unix/cs/promptKG/data/fewshots_example/wikidata/"

# ---- The 19 filenames from your screenshot ----
FILENAMES = [
    #"ont_1_movie_test.jsonl",
    "ont_2_music_test.jsonl",
    "ont_3_sport_test.jsonl",
    "ont_4_book_test.jsonl",
    "ont_5_military_test.jsonl",
    "ont_6_computer_test.jsonl",
    "ont_7_space_test.jsonl",
    "ont_8_politics_test.jsonl",
    "ont_9_nature_test.jsonl",
    "ont_10_culture_test.jsonl",
]

# ont_{index}_{category}_test.jsonl  ->  {index}_{category}_ontology.json, ont_{index}_{category}_few_shot.jsonl, ont_{index}_{category}_output.jsonl
PATTERN = re.compile(r"^ont_(\d+)_([a-z]+)_test\.jsonl$")

def make_paths(filename: str):
    m = PATTERN.match(filename)
    if not m:
        raise ValueError(f"Unexpected filename format: {filename}")
    idx, cat = m.groups()

    input_jsonl = os.path.join(BASE_INPUT, filename)
    ontology_json = os.path.join(BASE_ONTO, f"{idx}_{cat}_ontology.json")
    few_shot_jsonl = os.path.join(BASE_FEW, f"ont_{idx}_{cat}_few_shot.jsonl")

    # ont_{idx}_{cat}_test.jsonl -> ont_{idx}_{cat}_output.jsonl
    out_name = filename.replace("_test.jsonl", "_output.jsonl")
    output_jsonl = os.path.join(BASE_OUT, out_name)

    return input_jsonl, ontology_json, few_shot_jsonl, output_jsonl, f"ont_{idx}_{cat}"

# ---- Run all files ----
for fname in FILENAMES:
    try:
        INPUT_JSONL, ONTOLOGY_JSON, FEW_SHOT_JSONL, OUTPUT_JSONL, tag = make_paths(fname)

        print("\n" + "="*80)
        print(f"[RUN] {tag}")
        print("INPUT :", INPUT_JSONL)
        print("ONTO  :", ONTOLOGY_JSON)
        print("FEWS  :", FEW_SHOT_JSONL)
        print("OUTPUT:", OUTPUT_JSONL)

        out = run_pipeline(
            input_jsonl_path=INPUT_JSONL,
            ontology_json_path=ONTOLOGY_JSON,
            output_jsonl_path=OUTPUT_JSONL,
            max_items=None,          # keep as-is; change if you want to limit
            max_new_tokens=768,
            temperature=0.25,
            verbose=True,
            few_shot_jsonl_path=FEW_SHOT_JSONL,
        )

        print(f"[DONE] {tag}")
    except Exception as e:
        print(f"[ERROR] {fname}: {e}")



[RUN] ont_2_music
INPUT : /upb/users/b/balram/profiles/unix/cs/promptKG/data/input/wikidata/input_text/ont_2_music_test.jsonl
ONTO  : /upb/users/b/balram/profiles/unix/cs/promptKG/data/input/wikidata/input_ontology/2_music_ontology.json
FEWS  : /upb/users/b/balram/profiles/unix/cs/promptKG/data/fewshots_example/wikidata/ont_2_music_few_shot.jsonl
OUTPUT: /upb/users/b/balram/profiles/unix/cs/promptKG/data/output/prompt1/wikidata/ont_2_music_output.jsonl
[FEW-SHOT] Loaded 675 examples from: /upb/users/b/balram/profiles/unix/cs/promptKG/data/fewshots_example/wikidata/ont_2_music_few_shot.jsonl
⏳ Loading model: mistralai/Mistral-7B-Instruct-v0.3


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



✅ Done. Wrote 675 lines to: /upb/users/b/balram/profiles/unix/cs/promptKG/data/output/prompt1/wikidata/ont_2_music_output.jsonl | Total time: 3147.6s
[DONE] ont_2_music

[RUN] ont_3_sport
INPUT : /upb/users/b/balram/profiles/unix/cs/promptKG/data/input/wikidata/input_text/ont_3_sport_test.jsonl
ONTO  : /upb/users/b/balram/profiles/unix/cs/promptKG/data/input/wikidata/input_ontology/3_sport_ontology.json
FEWS  : /upb/users/b/balram/profiles/unix/cs/promptKG/data/fewshots_example/wikidata/ont_3_sport_few_shot.jsonl
OUTPUT: /upb/users/b/balram/profiles/unix/cs/promptKG/data/output/prompt1/wikidata/ont_3_sport_output.jsonl
[FEW-SHOT] Loaded 487 examples from: /upb/users/b/balram/profiles/unix/cs/promptKG/data/fewshots_example/wikidata/ont_3_sport_few_shot.jsonl
⏳ Loading model: mistralai/Mistral-7B-Instruct-v0.3


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

...

Device set to use cuda:0



✅ Done. Wrote 474 lines to: /upb/users/b/balram/profiles/unix/cs/promptKG/data/output/prompt1/wikidata/ont_9_nature_output.jsonl | Total time: 2541.0s
[DONE] ont_9_nature

[RUN] ont_10_culture
INPUT : /upb/users/b/balram/profiles/unix/cs/promptKG/data/input/wikidata/input_text/ont_10_culture_test.jsonl
ONTO  : /upb/users/b/balram/profiles/unix/cs/promptKG/data/input/wikidata/input_ontology/10_culture_ontology.json
FEWS  : /upb/users/b/balram/profiles/unix/cs/promptKG/data/fewshots_example/wikidata/ont_10_culture_few_shot.jsonl
OUTPUT: /upb/users/b/balram/profiles/unix/cs/promptKG/data/output/prompt1/wikidata/ont_10_culture_output.jsonl
[FEW-SHOT] Loaded 159 examples from: /upb/users/b/balram/profiles/unix/cs/promptKG/data/fewshots_example/wikidata/ont_10_culture_few_shot.jsonl
⏳ Loading model: mistralai/Mistral-7B-Instruct-v0.3


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cuda:0



✅ Done. Wrote 159 lines to: /upb/users/b/balram/profiles/unix/cs/promptKG/data/output/prompt1/wikidata/ont_10_culture_output.jsonl | Total time: 684.0s
[DONE] ont_10_culture
